In [2]:
import os
import numpy as np
import librosa
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Conv1DTranspose

clean_train_path = r'C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\clean_trainset_wav'
noisy_train_path = r'C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\noisy_trainset_wav'
clean_test_path = r'C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\clean_testset_wav'
noisy_test_path = r'C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\noisy_testset_wav'

In [3]:
def load_audio_files(path, max_length=16384):
    files = []
    for file_name in os.listdir(path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(path, file_name)
            audio, sr = librosa.load(file_path, sr=44100)
            audio = pad_or_truncate(audio, max_length)
            files.append(audio)
    return files

def pad_or_truncate(audio, max_length):
    if len(audio) > max_length:
        return audio[:max_length]
    elif len(audio) < max_length:
        return np.pad(audio, (0, max_length - len(audio)))
    else:
        return audio

# Load and preprocess clean and noisy training data
clean_train = load_audio_files(clean_train_path)
noisy_train = load_audio_files(noisy_train_path)

# Convert lists to numpy arrays and add channel dimension
clean_train = np.expand_dims(np.array(clean_train), axis=-1)
noisy_train = np.expand_dims(np.array(noisy_train), axis=-1)

print("Loaded and preprocessed training data:")
print(f"Clean train shape: {clean_train.shape}")
print(f"Noisy train shape: {noisy_train.shape}")

Loaded and preprocessed training data:
Clean train shape: (11572, 16384, 1)
Noisy train shape: (11572, 16384, 1)


In [4]:
# Define the denoising autoencoder model
input_audio = Input(shape=(16384, 1))
x = Conv1D(64, 3, activation='relu', padding='same')(input_audio)
x = Conv1D(32, 3, activation='relu', padding='same')(x)
encoded = Conv1D(64, 3, activation='relu', padding='same')(x)

x = Conv1DTranspose(64, 3, activation='relu', padding='same')(encoded)
x = Conv1DTranspose(32, 3, activation='relu', padding='same')(x)
decoded = Conv1DTranspose(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = Model(input_audio, decoded)
autoencoder.compile(optimizer='adam',
                     loss='mse',
                     metrics = 'accuracy')

# Train the autoencoder
autoencoder.fit(noisy_train, clean_train, epochs=10, batch_size=32, validation_split=0.2)

# Save the trained model
autoencoder.save('denoising_autoencoder.h5')



Epoch 1/10


290/290 [==============================] - 312s 1s/step - loss: 0.0095 - accuracy: 0.0000e+00 - val_loss: 1.8916e-05 - val_accuracy: 0.0000e+00
Epoch 2/10
290/290 [==============================] - 322s 1s/step - loss: 5.6440e-05 - accuracy: 0.0000e+00 - val_loss: 1.8916e-05 - val_accuracy: 0.0000e+00
Epoch 3/10
290/290 [==============================] - 320s 1s/step - loss: 5.6440e-05 - accuracy: 0.0000e+00 - val_loss: 1.8916e-05 - val_accuracy: 0.0000e+00
Epoch 4/10
290/290 [==============================] - 290s 1s/step - loss: 5.6440e-05 - accuracy: 0.0000e+00 - val_loss: 1.8916e-05 - val_accuracy: 0.0000e+00
Epoch 5/10
290/290 [==============================] - 290s 1s/step - loss: 5.6440e-05 - accuracy: 0.0000e+00 - val_loss: 1.8916e-05 - val_accuracy: 0.0000e+00
Epoch 6/10
290/290 [==============================] - 290s 1s/step - loss: 5.6440e-05 - accuracy: 0.0000e+00 - val_loss: 1.8916e-05 - val_accuracy: 0.0000e+00
Epoch 7/10
283/290 [==========================

KeyboardInterrupt: 

In [ ]:


# Load the trained model
autoencoder = tf.keras.models.load_model('denoising_autoencoder.h5')

# Load the new WAV file
new_audio, sr = librosa.load(r'C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Harsh.wav', sr=44100)

# Pad or truncate the new audio to the maximum length
new_audio = pad_or_truncate(new_audio, max_length=16384)

# Add a channel dimension to the new audio
new_audio = np.expand_dims(new_audio, axis=-1)

# Denoise the new audio using the trained model
denoised_audio = autoencoder.predict(new_audio)

# Convert the denoised audio back to a 1D array
denoised_audio = denoised_audio.squeeze(axis=-1)

# Display the waves before and after denoising
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(new_audio)
plt.title('Noisy Audio')

plt.subplot(2, 1, 2)
plt.plot(denoised_audio)
plt.title('Denoised Audio')

plt.show()